# Create Predict Price Model On Real Estate Data

## Connect to PostgreSQL

In [12]:
from pyspark.sql import SparkSession

jdbc_url = "jdbc:postgresql://postgres:5432/postgres"
table_name = "gold.gold_analytics_data"
connection_properties = {
    "user": "postgres",
    "password": "postgres"
}

## Import ML libraries

In [30]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline

## Load data

In [31]:
data = spark.read.jdbc(jdbc_url, table_name, properties=connection_properties)
data.printSchema()

root
 |-- sale_id: string (nullable = true)
 |-- account_id: integer (nullable = true)
 |-- ad_id: integer (nullable = true)
 |-- area: string (nullable = true)
 |-- area_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- latitude: decimal(10,6) (nullable = true)
 |-- longitude: decimal(10,6) (nullable = true)
 |-- location: string (nullable = true)
 |-- date: string (nullable = true)
 |-- price: long (nullable = true)
 |-- region_name: string (nullable = true)
 |-- rooms: integer (nullable = true)
 |-- size: decimal(10,2) (nullable = true)
 |-- type: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- price_per_sqm: decimal(33,11) (nullable = true)



## Data Preprocessing Pipeline

In [45]:
categorical_cols = ['area', 'area_name', 'category', 'category_name', 'location']
indexers = [StringIndexer(inputCol=col, outputCol=col + "_index").fit(data) for col in categorical_cols]
assembler = VectorAssembler(inputCols=['latitude', 'longitude', 'rooms', 'size', 'year', 'month', 'quarter'] + 
                                  [col + "_index" for col in categorical_cols], outputCol="raw_features")
scaler = StandardScaler(inputCol="raw_features", outputCol="features", withStd=True, withMean=True)
preprocessing_pipeline = Pipeline(stages=indexers + [assembler, scaler])
data_processed = preprocessing_pipeline.fit(data).transform(data)

## Split data into training and testing sets

In [46]:
train_data, test_data = data_processed.randomSplit([0.8, 0.2], seed=123)

## Initialize Gradient Boosted Trees Regressor

In [47]:
gbt = GBTRegressor(featuresCol="features", labelCol="price")

## Define parameter grid for hyperparameter tuning

In [48]:
param_grid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [5, 10]) \
    .addGrid(gbt.maxIter, [50, 100]) \
    .build()

## Initialize CrossValidator

In [49]:
crossval = CrossValidator(estimator=gbt,
                          estimatorParamMaps=param_grid,
                          evaluator=RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse"),
                          numFolds=3)

## Train the model

In [50]:
model = crossval.fit(train_data)

24/05/06 04:21:01 WARN DAGScheduler: Broadcasting large task binary with size 1001.3 KiB
24/05/06 04:21:01 WARN DAGScheduler: Broadcasting large task binary with size 1001.8 KiB
24/05/06 04:21:01 WARN DAGScheduler: Broadcasting large task binary with size 1002.3 KiB
24/05/06 04:21:01 WARN DAGScheduler: Broadcasting large task binary with size 1003.5 KiB
24/05/06 04:21:02 WARN DAGScheduler: Broadcasting large task binary with size 1005.7 KiB
24/05/06 04:21:02 WARN DAGScheduler: Broadcasting large task binary with size 1008.4 KiB
24/05/06 04:21:02 WARN DAGScheduler: Broadcasting large task binary with size 1008.9 KiB
24/05/06 04:21:02 WARN DAGScheduler: Broadcasting large task binary with size 1009.4 KiB
24/05/06 04:21:02 WARN DAGScheduler: Broadcasting large task binary with size 1010.6 KiB
24/05/06 04:21:02 WARN DAGScheduler: Broadcasting large task binary with size 1012.8 KiB
24/05/06 04:21:02 WARN DAGScheduler: Broadcasting large task binary with size 1015.5 KiB
24/05/06 04:21:02 WAR

24/05/06 04:23:34 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
24/05/06 04:23:34 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
24/05/06 04:23:34 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
24/05/06 04:23:34 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
24/05/06 04:23:35 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
24/05/06 04:23:35 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
24/05/06 04:23:36 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
24/05/06 04:23:36 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
24/05/06 04:23:36 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
24/05/06 04:23:37 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
24/05/06 04:23:37 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
24/05/06 04:23:37 WARN DAGScheduler: Broadcasting larg

24/05/06 04:24:16 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
24/05/06 04:24:17 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB


24/05/06 04:24:17 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
24/05/06 04:24:17 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
24/05/06 04:24:18 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
24/05/06 04:24:18 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
24/05/06 04:24:18 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
24/05/06 04:24:19 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
24/05/06 04:24:19 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
24/05/06 04:24:19 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
24/05/06 04:24:19 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
24/05/06 04:24:20 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
24/05/06 04:24:20 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
24/05/06 04:24:20 WARN DAGScheduler: Broadcasting larg

24/05/06 04:24:40 WARN DAGScheduler: Broadcasting large task binary with size 5.6 MiB
24/05/06 04:24:40 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB
24/05/06 04:24:41 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB
24/05/06 04:24:41 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB
24/05/06 04:24:42 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB
24/05/06 04:24:42 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB
24/05/06 04:24:42 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB
24/05/06 04:24:43 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB
24/05/06 04:24:43 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB
24/05/06 04:24:43 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB
24/05/06 04:24:44 WARN DAGScheduler: Broadcasting large task binary with size 5.8 MiB
24/05/06 04:24:44 WARN DAGScheduler: Broadcasting larg

...

24/05/06 04:41:55 WARN DAGScheduler: Broadcasting large task binary with size 8.6 MiB
24/05/06 04:41:55 WARN DAGScheduler: Broadcasting large task binary with size 8.7 MiB
24/05/06 04:41:56 WARN DAGScheduler: Broadcasting large task binary with size 8.7 MiB


24/05/06 04:41:56 WARN DAGScheduler: Broadcasting large task binary with size 8.7 MiB
24/05/06 04:41:57 WARN DAGScheduler: Broadcasting large task binary with size 8.7 MiB
24/05/06 04:41:57 WARN DAGScheduler: Broadcasting large task binary with size 8.7 MiB
24/05/06 04:41:57 WARN DAGScheduler: Broadcasting large task binary with size 8.7 MiB
24/05/06 04:41:58 WARN DAGScheduler: Broadcasting large task binary with size 8.7 MiB
24/05/06 04:41:58 WARN DAGScheduler: Broadcasting large task binary with size 8.7 MiB
24/05/06 04:41:59 WARN DAGScheduler: Broadcasting large task binary with size 8.7 MiB
24/05/06 04:41:59 WARN DAGScheduler: Broadcasting large task binary with size 8.7 MiB
24/05/06 04:42:00 WARN DAGScheduler: Broadcasting large task binary with size 8.7 MiB
24/05/06 04:42:00 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB
24/05/06 04:42:01 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB
24/05/06 04:42:01 WARN DAGScheduler: Broadcasting larg

24/05/06 04:42:06 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB
24/05/06 04:42:06 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB
24/05/06 04:42:07 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB
24/05/06 04:42:07 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB
24/05/06 04:42:08 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB
24/05/06 04:42:08 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB
24/05/06 04:42:08 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB
24/05/06 04:42:09 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB
24/05/06 04:42:09 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB
24/05/06 04:42:10 WARN DAGScheduler: Broadcasting large task binary with size 9.0 MiB
24/05/06 04:42:10 WARN DAGScheduler: Broadcasting large task binary with size 9.0 MiB
24/05/06 04:42:11 WARN DAGScheduler: Broadcasting larg

24/05/06 04:42:14 WARN DAGScheduler: Broadcasting large task binary with size 9.0 MiB
24/05/06 04:42:15 WARN DAGScheduler: Broadcasting large task binary with size 9.1 MiB
24/05/06 04:42:15 WARN DAGScheduler: Broadcasting large task binary with size 9.1 MiB
24/05/06 04:42:16 WARN DAGScheduler: Broadcasting large task binary with size 9.1 MiB
24/05/06 04:42:16 WARN DAGScheduler: Broadcasting large task binary with size 9.1 MiB
24/05/06 04:42:17 WARN DAGScheduler: Broadcasting large task binary with size 9.1 MiB
24/05/06 04:42:17 WARN DAGScheduler: Broadcasting large task binary with size 9.1 MiB
24/05/06 04:42:18 WARN DAGScheduler: Broadcasting large task binary with size 9.1 MiB
24/05/06 04:42:18 WARN DAGScheduler: Broadcasting large task binary with size 9.1 MiB
24/05/06 04:42:18 WARN DAGScheduler: Broadcasting large task binary with size 9.2 MiB
24/05/06 04:42:19 WARN DAGScheduler: Broadcasting large task binary with size 9.2 MiB
24/05/06 04:42:19 WARN DAGScheduler: Broadcasting larg

24/05/06 04:42:21 WARN DAGScheduler: Broadcasting large task binary with size 9.2 MiB
24/05/06 04:42:22 WARN DAGScheduler: Broadcasting large task binary with size 9.2 MiB
24/05/06 04:42:22 WARN DAGScheduler: Broadcasting large task binary with size 9.2 MiB
24/05/06 04:42:22 WARN DAGScheduler: Broadcasting large task binary with size 9.2 MiB
24/05/06 04:42:23 WARN DAGScheduler: Broadcasting large task binary with size 9.2 MiB
24/05/06 04:42:23 WARN DAGScheduler: Broadcasting large task binary with size 9.3 MiB
24/05/06 04:42:24 WARN DAGScheduler: Broadcasting large task binary with size 9.3 MiB
24/05/06 04:42:24 WARN DAGScheduler: Broadcasting large task binary with size 9.3 MiB
24/05/06 04:42:25 WARN DAGScheduler: Broadcasting large task binary with size 9.3 MiB
24/05/06 04:42:25 WARN DAGScheduler: Broadcasting large task binary with size 9.3 MiB
24/05/06 04:42:26 WARN DAGScheduler: Broadcasting large task binary with size 9.3 MiB
24/05/06 04:42:26 WARN DAGScheduler: Broadcasting larg

## Make predictions

In [53]:
predictions = model.transform(test_data)

## Evaluate the model

In [54]:
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 4.59919e+10


## Bruh, this model is so bad, I dont know why @@